In [ ]:
%run ../Helper/Setup.ipynb

In [ ]:
def import_data(path):
    df = pd.read_csv(path)
    return df

In [ ]:
data = import_data('../../results/hardware/optimized/processed.csv')

In [ ]:
def rename(name):
    split = name.split('-')
    if len(split) == 2:
        return f'{split[0]}*'
    return name


data['server'] = data['server'].apply(lambda x: rename(x))
data['client'] = data['client'].apply(lambda x: rename(x))

In [ ]:
def plot_goodput(df):

    implementations = pd.unique(df[['server', 'client']].values.ravel('K'))
    labels = df.hostpair.unique()

    implementation_order = {
        'lsquic': 1,
        'lsquic*': 2,
        'quiche': 3,
        'quiche*': 4,
        'tcp+tls': 5,
    }
    
    hostpair_order = {
        'tinyman-idex': 7,
        'uniswap-solana': 5,
        'dogecoin-dogecoincash': 3,
        'goracle-gard': 6,
        'amdexp0-amdexp1': 4,
    }
    
    implementations = sorted(implementations, key=lambda x: implementation_order.get(x, 1000))
    labels = sorted(labels, key=lambda x: hostpair_order.get(x, 1000))

    lable_names = {
        'tinyman-idex': 'Intel-3',
        'uniswap-solana': 'Intel-2',
        'dogecoin-dogecoincash': 'Intel-1',
        'goracle-gard': 'AMD-2',
        'amdexp0-amdexp1':  'AMD-1',
    }
    
    
    implementation_names = {
        'lsquic': 'LSQUIC',
        'lsquic*': 'LSQUIC*',
        'quiche': 'quiche',
        'quiche*': 'quiche*',
        'quich': 'quiche',
        'tls1.3': 'TCP/TLS',
        'tcp+tls': 'TCP/TLS',
    }
        
    colors = [ColorDarkerBlue, ColorDarkerBlue, ColorGreen, ColorGreen, ColorRed]
    patterns = [" ", " ", " ", " ", " " ]
    linestyles = ['-', '-', '-', '-', '-']
    markers = ['^', '^', 'o', 'o', 's']
    
    facecolors= [ColorDarkerBlue, 'white', ColorGreen, 'white', ColorRed]

    
    def style(i):
        i = i % len(colors)
        return {
            #"boxprops": dict(facecolor=c, color=c),
            "showmeans": True,
            "boxprops": dict(color=colors[i], linestyle=linestyles[i], alpha=0),
            "capprops": dict(color=colors[i], alpha=0),
            "whiskerprops": dict(alpha=0),
            "flierprops": dict(color=colors[i], markeredgecolor=colors[i]),
            #"medianprops": dict(color=c),
            #"meanprops": dict(color=c),
            "medianprops": dict(color=colors[i], alpha=0),
            "meanprops": dict(markerfacecolor=facecolors[i], markeredgecolor=colors[i], marker=markers[i], markersize=7),
            "whis": [5,95],
            "showfliers": False,
            "widths": .22,
            "positions": [2 * x - 0.7 + 0.35 * i for x in range(len(labels))],
        }
    
    width, height = set_size('single')
    
    fig, ax1 = plt.subplots(figsize=(width, height*0.8))
    plot_shades(labels, ax1, 1, factor=2)
    
    for idx, imp in enumerate(implementations):
        
        d = df.loc[(df.server == imp) & (df.client == imp)]
        bp1 = ax1.boxplot([d[d.hostpair == c].goodput for c in labels], labels=labels,
            #hatch=patterns[0],
            **style(idx),
           )
    
    ax1.autoscale()
    
    
    ax1.set(
        xticks=list(map(lambda x: x * 2, range(len(labels)))), 
        xticklabels=[lable_names[c] for c in labels]
    )
    #ax1.set(yticks=[2000, 4000, 6000, 8000,])

#    for i in range(len(labels))[1::2]:
#        ax1.axvspan(-1 + 2 * i , 1 + 2 * i, alpha=0.1, color='grey', lw=0)    

    
    ax1.set_ylabel('Goodput [Gbit/s]')
    ax1.set_ylim(0)
    
    mkfunc = lambda x, pos: f'{(x * 1e-3):.0f}'
    mkformatter = matplotlib.ticker.FuncFormatter(mkfunc)
    ax1.yaxis.set_major_formatter(mkformatter)
    
    legend_elements = [
        #Patch(facecolor='white', edgecolor=colors[i], linestyle=linestyles[i], label=implementation_names[imp])
        mlines.Line2D([], [], color=colors[i], markerfacecolor=facecolors[i], marker=markers[i], linestyle='None', label=implementation_names[imp])
        for i, imp in enumerate(implementations)
    ]
    
    
    ax1.legend(handles=legend_elements, loc="upper left",
               ncol=3,
               #bbox_to_anchor=(0, 1.01)
              )
    fig.tight_layout()
    fig.savefig('hardware.pdf')

    

In [ ]:
plot_goodput(data)